# 1) Double Selection Sort (a.k.a. Bidirectional Selection)

**Intuition (what changes):**
Classic selection does **one** scan of the unsorted suffix to place the **minimum** at the front.
**Double selection** does **one** scan to track **both** the minimum and maximum, then writes the min to the left end and the max to the right end. That shrinks the unsorted window from **both** sides.

**Loop invariant:**
After pass $i$ (0-indexed),

* positions $[0..i-1]$ contain the $i$ smallest elements in nondecreasing order, and
* positions $[n-i..n-1]$ contain the $i$ largest elements in nonincreasing order,
* the subarray $[i..n-1-i]$ is the active window.

**Worked example**

Start: `A = [ 4, 9, 1, 7, 2 ]`, $n=5$

```
Pass i=0, active window [0..4]
Scan once, track (min, max):
  seen 4 → min=4@0, max=4@0
  seen 9 → max=9@1
  seen 1 → min=1@2
  seen 7 → (no change)
  seen 2 → (no change)
Write ends:
  swap A[0] ↔ A[2]   -> [ 1, 9, 4, 7, 2 ]
  (fix index if max was at i) here max at 1
  swap A[4] ↔ A[1]   -> [ 1, 2, 4, 7, 9 ]

Pass i=1, active window [1..3]
Scan once, track (min, max) over [2,4,7]:
  min=2@1, max=7@3 → ends already correct → no swaps

Done: [ 1, 2, 4, 7, 9 ]
```

**Pseudocode**

```text
for i = 0 to ⌊(n-1)/2⌋:
    lo = i
    hi = n-1-i
    min_idx = lo
    max_idx = lo
    for j = lo+1 to hi:
        if A[j] < A[min_idx]:
            min_idx = j
        elif A[j] > A[max_idx]:
            max_idx = j
    swap A[lo] ↔ A[min_idx]
    # if we moved the previous max by swapping the min, repair its index
    if max_idx == lo: max_idx = min_idx
    swap A[hi] ↔ A[max_idx]
```

**Cost & notes**

* Comparisons per pass over window length $k$: worst-case $\approx 2(k-1)$; with the “compare-to-min then maybe compare-to-max” trick, average $\approx \tfrac{3}{2}(k-1)$.
* Total comparisons $\Theta(n^2)$; fewer **passes** than classic selection (roughly half), but same asymptotics.
* In-place, $O(1)$ extra space. Not stable (swaps can cross equal keys).

---

# 2) Stable Selection Sort (selection + stable insert)

**Intuition (what changes):**
Selection sort isn’t stable because swapping the found minimum into position $i$ can jump over equal elements.
**Stable selection** still finds the min in the suffix, but **removes** it and **inserts** it at $i$, shifting the block $[i..m-1]$ one step to the right. That preserves the order of equal keys.

**Loop invariant:**
After pass $i$, prefix $[0..i]$ holds the $i{+}1$ smallest elements **in original relative order** (stability), suffix remains arbitrary.

**Why the insert is stable (one-liner):**
We only shift the contiguous block $[i..m-1]$ to the right by one; internal order of that block is unchanged, so equals keep their original order.

**Worked example (with equal keys labeled)**

```
A = [ 4a, 4b, 2, 4c ]

i=0: scan [0..3] → min=2 at m=2
key = A[2]
shift right: [ 4a, 4b, 4b, 4c ] (conceptually; we write backwards)
final after insert at i:
A = [ 2, 4a, 4b, 4c ]  (4a before 4b before 4c — preserved ✅)

i=1: scan [1..3] → already in order
```

**Pseudocode**

```text
for i = 0 to n-1:
    # find min index m in A[i..n-1]
    m = i
    for j = i+1 to n-1:
        if A[j] < A[m]: m = j
    # stable insert: store min, shift A[i..m-1] one step to the right
    key = A[m]
    while m > i:
        A[m] = A[m-1]
        m -= 1
    A[i] = key
```

**Cost & notes**

* Comparisons to find mins: $\sum_{k=1}^{n-1} k = \tfrac{n(n-1)}{2} = \Theta(n^2)$.
* Moves due to shifting: again $\Theta(n^2)$ in the worst case.
* Space $O(1)$. **Stable by construction.**

---

# 3) Tournament (Binary) Sort

**Intuition (what changes):**
Think of the array as players in a **knock-out tournament**. Each match compares two players; the winner (smaller key) goes up. The **root** is the global minimum. After extracting the winner, we only **replay** the matches on the path from its leaf to the root to find the next winner.

You can implement the tournament as a complete binary tree stored in arrays.

**Structure**

* Leaves store the elements (or indices).
* Each internal node stores the **winner** (index/value) of its two children.
* On ties, break by **smaller original index** → this makes the algorithm **stable**.

**One build + many replays**

* Build once from all leaves bottom-up (this already performs the comparisons that find the initial minimum).
* Extract minimum $n$ times:

  1. Output the root’s winner.
  2. Replace its leaf with $+\infty$ (a sentinel).
  3. **Replay** matches from that leaf up to the root (height $\lceil \log_2 n \rceil$ comparisons).

**ASCII (6 elements)**

Array: `A = [ 7, 3, 8, 2, 5, 6 ]`

```
Leaves (indices 0..5)

          (root)
          (  2  )  ← winner is value 2@idx=3
         /       \
      ( 3 )     ( 2 )
     /    \     /    \
   (7)   (3)  (8)   (2)
                    /  \
                  (5)  (6)

Extract 2:
- Set leaf idx=3 := +∞
- Replay matches along its path to root to get next global min (3)
- Repeat...
```

**Pseudocode (index-based, stable ties)**

```text
# Assume N leaves in array L[0..N-1]
# Internal nodes are stored in T[1..M], T[1] is root (store index of winner)
# For simplicity, imagine a perfect tree; in practice, pad with +∞ leaves.

build():
    put leaf indices 0..N-1 at the bottom level
    for each internal node bottom-up:
        T[node] = better(T[left], T[right])   # tie → smaller index

extract_min():
    i = T[1]                  # winner index at root
    output L[i]
    L[i] = +∞                 # deactivate winner
    # replay path from leaf-of(i) to root:
    node = leaf_node_of(i)
    while node > 1:
        node = parent(node)
        T[node] = better(T[left(node)], T[right(node)])
    return i
```

**Cost & notes**

* Build (bottom-up): $O(n)$ space, $\approx n-1$ comparisons.
* Each extraction: $O(\log n)$ comparisons (tree height).
* Total: $O(n \log n)$ time, $O(n)$ extra space.
* **Stability:** ensure `better(a,b)` breaks ties by **original index**.

---

## Complexity & Behavior Cheat-Sheet

| Algorithm                | Stable | Time (worst)      | Extra space | Writes               | Notes                               |
| ------------------------ | ------ | ----------------- | ----------- | -------------------- | ----------------------------------- |
| Double Selection Sort    | No     | $\Theta(n^2)$     | $O(1)$      | $O(n)$ swaps         | Fewer passes than classic selection |
| Stable Selection Sort    | Yes    | $\Theta(n^2)$     | $O(1)$      | $\Theta(n^2)$ shifts | Selection + stable insert           |
| Tournament (Binary) Sort | Yes\*  | $\Theta(n\log n)$ | $O(n)$      | $O(n)$               | \*Stable if tie breaks by index     |